# Example application of tool

Let's apply the tool to a set of variants from a cancer cell line.

Dataset is from [Talsania et. al. 2022](https://genomebiology.biomedcentral.com/articles/10.1186/s13059-022-02816-6). These are structural variants called using the [Manta pipeline](https://github.com/Illumina/manta) on WGS data from Illumina in HCC1395 tumor cells.

In [1]:
import pandas as pd
import numpy as np 
import os



from pathlib import Path

# Get input variants

In [2]:
# Get variants

ftp_path = 'https://ftp-trace.ncbi.nlm.nih.gov/ReferenceSamples/seqc/Somatic_Mutation_WG/analysis/SVs/VCFs/tumor.illumina.manta.EA_T_1.vcf.gz'
in_file = '../test/tumor.illumina.manta.EA_T_1.vcf'

if not Path(in_file).is_file():
    os.system(f'wget -P ../test/ {ftp_path}')
    os.system(f'gupzip {in_file}.gz')


In [4]:
# Read input

import sys
sys.path.insert(0, '../scripts')

import reading_utils
reading_utils.var_set_size = 2000

variants = reading_utils.read_input(in_file, 0)
variants

,CHROM,POS,END,REF,ALT,SVTYPE,SVLEN
0,chr1,1117831,NaN,C,[chr22:20272153[C,BND,NaN
1,chr1,1119512,NaN,A,ACAGTGC]chr22:20302979],BND,NaN
2,chr1,3721048,3734333,T,<DUP:TANDEM>,DUP,13285
3,chr1,6742482,NaN,C,C[chr12:96100887[,BND,NaN
4,chr1,9357666,9377061,G,<DEL>,DEL,-19395
...,...,...,...,...,...,...,...
1543,chrY,14531089,NaN,T,T[chrX:6219006[,BND,NaN
1544,chrY,14533586,NaN,A,A]chrX:6219008],BND,NaN
1545,chr16_KI270728v1_random,1769992,NaN,A,A[chr9:129442980[,BND,NaN
1546,chr17_KI270729v1_random,162446,162616,GAGTCCATTCGATGATTTCATTAGATTCCATTGGAAGATGATTCCA...,G,DEL,-170


In [14]:
# Look at varinat types present
variants.SVTYPE.value_counts()

DEL    552
DUP    294
INS     80
Name: SVTYPE, dtype: int64

# Score variants using tool

In [6]:
# Get run command

file = 'tumor'#'CTCF_del' # Output files prefix
directory = 'test/tumor_output'# '../test/output' # Output directory

print('Run this command in the main directory:\n')
print('python scripts/score_var.py', in_file,
      '--file', file, # File name prefix for outputs
      '--dir', directory, # Path to save output in 
      '--augment', # Get the average augmented scores
      '--get_scores') # Get disruption scores


Run this command in the main directory:

python scripts/score_var.py test/tumor.illumina.manta.EA_T_1.vcf --file tumor --dir test/tumor_output --augment --get_scores


In [7]:
# Get path to output files
out_file = os.path.join(directory, file)

# Prioritize most disruptive variants

In [9]:
# Read scores output

scores = pd.read_csv(f'../{out_file}_scores', sep = '\t')
scores.iloc[:10,:10]

,var_index,mse_mean,corr_mean
0,0,NaN,NaN
1,1,NaN,NaN
2,2,0.024681,0.955341
3,3,NaN,NaN
4,4,0.015164,0.973157
5,5,NaN,NaN
6,6,NaN,NaN
7,7,0.009095,0.964207
8,8,0.012784,0.964684
9,9,NaN,NaN


In [11]:
# Add scores dataframe to input dataframe to match them with information about the perturbations

scores = pd.concat([variants, scores], axis = 1)
scores.iloc[:10,:10]

,CHROM,POS,END,REF,ALT,SVTYPE,SVLEN,var_index,mse_mean,corr_mean
0,chr1,1117831,NaN,C,[chr22:20272153[C,BND,NaN,0,NaN,NaN
1,chr1,1119512,NaN,A,ACAGTGC]chr22:20302979],BND,NaN,1,NaN,NaN
2,chr1,3721048,3734333,T,<DUP:TANDEM>,DUP,13285,2,0.024681,0.955341
3,chr1,6742482,NaN,C,C[chr12:96100887[,BND,NaN,3,NaN,NaN
4,chr1,9357666,9377061,G,<DEL>,DEL,-19395,4,0.015164,0.973157
5,chr1,9495567,36665592,T,<DUP:TANDEM>,DUP,27170025,5,NaN,NaN
6,chr1,13349505,NaN,A,A]chr8:101508951],BND,NaN,6,NaN,NaN
7,chr1,13931876,13932299,TGCCCAGGCTGGAGTGCAGTGGCACGATCTTGGCTCACTGCAACCT...,T,DEL,-423,7,0.009095,0.964207
8,chr1,17393975,17394067,GATGAGATGGCCTTCTGCTACACCCAGGCTCCCCACAAGACAACGT...,G,DEL,-92,8,0.012784,0.964684
9,chr1,18359587,NaN,A,A]chr6:163603712],BND,NaN,9,NaN,NaN


In [24]:
# Get top 3 most disruptive variants 

top_variants = []

for method in ['mse_mean', 'corr_mean']:
    
    for SVTYPE in [x for x in scores.SVTYPE.unique() if x is not np.nan]:

        top_var = list(scores[(scores.SVTYPE == SVTYPE) & (~np.isnan(scores[method]))]
                            .sort_values(method, ascending = False)
                            .head(3)
                            .index
                            .values)

        for var in top_var:
            top_variants.append(var)
        
top_variants[:10]

[829, 64, 549, 25, 894, 1355, 439, 421, 1172, 397]

# Get maps for top scoring variants using tool

In [23]:
variants[[x in top_variants for x in variants.index]][:10]

,CHROM,POS,END,REF,ALT,SVTYPE,SVLEN
25,chr1,58064887,58560456,T,<DEL>,DEL,-495569
64,chr1,149799777,150031617,G,<DUP:TANDEM>,DUP,231840
397,chr6,1233843,1233907,A,<DUP:TANDEM>,DUP,64
398,chr6,1233870,1233870,A,AGAGGAGGCTGTCGGGGGACCGGGAGGACCTCGCCTGCTGTCCATG...,INS,64
403,chr6,4259990,4260069,A,<DUP:TANDEM>,DUP,79
421,chr6,17874942,17874942,T,TAATTCATTCCTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTGAG...,INS,323
439,chr6,23978115,23978115,G,GAAAAATGACAACCAGGCCGGGTGCGGTGGCTCACGCCTGTAATCC...,INS,323
549,chr6,72060264,72637247,C,<DUP:TANDEM>,DUP,576983
607,chr6,168695904,168696064,GCCATTCAGATCATATTTCATGGAGGCCAGGTGTGCTGTGGAATGT...,G,DEL,-160
829,chr10,4834359,5133665,A,<DUP:TANDEM>,DUP,299306


In [26]:
# Get run command

file_top = 'tumor_top'# Output files prefix
directory_top = 'test/tumor_output'# Output directory

print('Run this command in the main directory:\n')
print('python scripts/score_var.py', in_file,
      '--file', file, # File name prefix for outputs
      '--dir', directory, # Path to save output in 
      '--augment', # Get the average augmented scores
      '--get_scores', # Get disruption scores
      'get_maps', # Get predicted contact frequency maps
      'get_tracks') # Get disruption tracks


Run this command in the main directory:

python scripts/score_var.py test/tumor.illumina.manta.EA_T_1.vcf --file tumor_top --dir test/tumor_output --augment --get_scores get_maps get_tracks


In [27]:
# Get path to output files
out_file_top = os.path.join(directory_top, file_top)

# Plot maps for top scoring variants